In [1]:
import json, os
import numpy as np
import pandas as pd
import os, time, datetime, pickle

from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Convolution1D, MaxPooling1D


Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


### Data Loading

In [2]:
# Data Preparation
# ==================================================
# Loads x/y from pickle, assumes pre-shuffled
datapath = './mapped_data/'
datafiles = [f for f in os.listdir(datapath) if '.pkl' in f]
filename = datafiles[0]
with open(datapath + filename, 'r') as f:
    dataset = pickle.load(f)
    x_shuffled = dataset['x']
    y_shuffled = dataset['y']


# Split train/test set
split_sample_index = -1 * int(.1 * float(len(y_shuffled)))
X_train, X_test = x_shuffled[:split_sample_index], x_shuffled[split_sample_index:]
y_train, y_test = y_shuffled[:split_sample_index], y_shuffled[split_sample_index:]

In [3]:
print(X_train.shape)

(540000, 71)


### Example input data and label

In [ ]:
#Example of input data
example = x_shuffled[0]
print(example.shape)
print(example)
print(y_shuffled[0])

(71,)
[   290    290   9294   5976    374 115081   6093 110179   2146 113007
 110594    290   1978 113376 110693      9      9      9      9      9
      9      9      9      9      9      9      9      9      9      9
      9      9      9      9      9      9      9      9      9      9
      9      9      9      9      9      9      9      9      9      9
      9      9      9      9      9      9      9      9      9      9
      9      9      9      9      9      9      9      9      9      9
      9]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


### Model Creation

In [ ]:
# Embedding
max_features = np.max(x_shuffled)
maxlen = max([len(t) for t in x_shuffled])
embedding_size = 128

# Convolution
filter_length = 5
nb_filter = 64
pool_length = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 30
nb_epoch = 1

#For user in final fully connected layer
classes = len(y_shuffled[0]) 

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen))
model.add(Dropout(0.25))
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
model.add(MaxPooling1D(pool_length=pool_length))
model.add(LSTM(lstm_output_size))
model.add(Dense(classes))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          validation_data=(X_test, y_test))
score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 540000 samples, validate on 60000 samples
Epoch 1/1
  2070/540000 [..............................] - ETA: 4333s - loss: 0.2759 - acc: 0.9380